In [1]:
from prompt import add_user_message, add_assistant_message, chat
from pprint import pp
messages = []
add_user_message(messages, "What's the weather in San Francisco, California?")
answer = chat(messages)
answer

'I am not able to provide real-time weather information. To get current weather conditions in San Francisco, I recommend:\n\n1. Checking a weather website or app\n2. Looking up the local weather forecast\n3. Checking the National Weather Service website'

## Tool Use

1.Initial Request: You send Claude a question along with instructions on how to get extra data from external sources

2.Tool Request: Claude analyzes the question and decides it needs additional information, then asks for specific details about what data it needs

3.Data Retrieval: Your server runs code to fetch the requested information from external APIs or databases

4.Final Response: You send the retrieved data back to Claude, which then generates a complete response using both the original question and the fresh data

**for example**:

1.使用者提出問題（例如：「今天天氣如何？」）

2.LLM 判斷需要呼叫特定工具（function），例如「get_weather」來獲取相關資訊

3.伺服器端收到這個呼叫需求，並且用相應工具查詢資料

4.工具結果回傳給 LLM，LLM 根據結果生成回答送回給使用者